# Split filelist file into train and test sets

Use a train ratio or number of samples in test set.


In [2]:
# Load the data from the csv file
import pandas as pd
import os
import random

random.seed(42)

dataset_name = "Almaz_new"
data: pd.DataFrame = pd.read_csv(f"../filelists/{dataset_name}.csv",sep='\t')
#data = data.drop(['Unnamed: 0.3','Unnamed: 0.1','Unnamed: 0.2','Unnamed: 0'],axis=1)
#data['phonemes'] = data['phoneme']
#data = data.drop(['phoneme'],axis=1)
print(data.head())
print(type(data['uttid'][0]))

   Unnamed: 0  spkidx  spkid   txtid   uttid  \
0           0       0      1   i2643   i2643   
1           1       0      1   i2124   i2124   
2           2       0      1    i574    i574   
3           3       0      1  i02299  i02299   
4           4       0      1    i248    i248   

                                                text  \
0  аны сәяси совет секретаре тр дәүләт советы рәи...   
1  аны зурга күтәреп җитди эш дип саныйсы да юк  ...   
2                      аңардан сак булырга кирәк иде   
3  ләкин сез күз алдыгызга китерә аласызмы минем ...   
4  стариков белән газинур аның һәр кул хәрәкәтен ...   

                                            phonemes  
0  ɒnˈɯ sæjæsˈi sowˈet sekretɑrˈe tˌeˈer dæwlˈæt ...  
1  ɒnˈɯ zurʁˈɑ kytærˈep ʑitdˈi ˈeʃ dˈip sɒnɯjsˈɯ ...  
2                 ɒŋɒrdˈɑn sˈɒq buɫɯrʁˈɑ kirˈæk idˈe  
3  lækˈin sˈez kˈyz ɒɫdɯʁɯzʁˈɑ kiterˈæ ɒɫɒsɯzmˈɯ ...  
4  stɒriqˈow belˈæn ʁɒzinˈur ɒnˈɯŋ hˈær qˈuɫ xæræ...  
<class 'str'>


In [2]:
#data.loc[data['uttid'] == 1462]
#data = data.drop([3107,3481],axis=0)

In [3]:
# Support for DataFrames
def split_file_list(orig_data: pd.DataFrame, train_ratio=None, test_samples=None, max_samples=None):
    # Shuffle the data
    data = orig_data.sample(frac=1).reset_index(drop=True)

    if max_samples is not None:
        data = data[:max_samples]

    if test_samples is not None:
        train_set = data[:-test_samples]
        test_set = data[-test_samples:]
    elif train_ratio is not None:
        train_set_size = int(len(data) * train_ratio)
        train_set = data[:train_set_size]
        test_set = data[train_set_size:]

    else:
        raise ValueError("Either 'train_ratio' or 'test_samples' should be provided.")

    return train_set, test_set


# Example usage
train_data, val_data = split_file_list(data, test_samples=240)

### Save phonemes and text of train_data, val_data


In [4]:
i_dir = "home/ips/projects/tatar-tts-2/VITS/vits/Almaz"
o_file_train = f"../filelists/{dataset_name}_audio_sid_text_train_filelist.txt"
o_file_val = f"../filelists/{dataset_name}_audio_sid_text_test_filelist.txt"

link_name = "DUMMY5"

In [5]:
def create_path_map(source_dir):
    path_map = {}
    for files in os.walk(source_dir):
        !pwd
        for file in files:
            if file.endswith(".wav"):
                path_map[file] = os.path.join(root, file)
                print(path_map[file])
    return path_map


def save_file_list(data, out_file_path, source_dir, path_map, link_name, cleaned_text=False):
    with open(out_file_path, "w") as file:
        for row in data.itertuples():
            uttid = f"{row.uttid}.wav"
            path = path_map[uttid].replace(source_dir, link_name)
            spkidx = row.spkidx
            info = row.text if not cleaned_text else row.phonemes

            file.write(f"{path}|{spkidx}|{info}\n")
            # Print every nth sample
            if row.Index % 5000 == 0:
                print(f"{row.Index}: {path}|{spkidx}|{info}")

    print(f"Saved to '{out_file_path}' ({len(data)} samples).")


def save_files(data, out_file_path, source_dir, path_map, link_name):
    save_file_list(train_data, out_file_path, source_dir, path_map, link_name)
    if "phonemes" in data.columns:
        out_file_path = out_file_path.replace(".txt", ".txt.cleaned")
        save_file_list(data, out_file_path, source_dir,
                       path_map, link_name, cleaned_text=True)

In [6]:
path_map = {}
for files in os.walk(i_dir):
    for file in files:
        if file.endswith(".wav"):
            path_map[file] = os.path.join(root, file)

In [7]:
path_map = create_path_map(i_dir)
print(path_map)

#save_files(train_data, o_file_train, i_dir, path_map, link_name)
#save_files(val_data, o_file_val, i_dir, path_map, link_name)

{}


### Create a symlink to the dataset


In [ ]:
# Create symlink to the dataset
!ln -s {i_dir} {link_name}